# DLW Practical 3 (extra)
# Any architecture on COLOR

**Introduction**

In this practical, we will apply a linear model to the COLOR data set. The COLOR data set comes as 18 different colour classes (red, green, gold, etc.) consisting of 9x9x3 RGB images.

The challenge is to build a classifier that can correctly classify each of the 18 different colour classes present in the test set. You are welcome to use any network architecture for this exercise. Use a confusion matrix to understand what the model is struggling to classify correctly. What is your best test set accuracy?



In [8]:
%matplotlib inline

import os
import time
import itertools
import datetime
import pandas as pd
import pickle as pk
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile
from tensorflow.python.framework import tensor_shape
from tensorflow.contrib.tensorboard.plugins import projector

with open('data.pk', 'rb') as handle:
    data = pk.load(handle)

print('\nTraining: {}\nValidation: {}\nTesting: {}\nUnique labels: {}\n'.format(data[0].shape, data[1].shape, data[2].shape, data[3]))

col = len(data[0].columns.values) - 3

train_x, train_y = data[0][list(range(col))].values, pd.get_dummies(data[0]['class_id'].values).as_matrix()
val_x, val_y = data[1][list(range(col))].values, pd.get_dummies(data[1]['class_id'].values).as_matrix()
test_x, test_y = data[2][list(range(col))].values, pd.get_dummies(data[2]['class_id'].values).as_matrix()

train_examples, x_dim, n_classes = train_x.shape[0], train_x.shape[1], train_y.shape[1]


FileNotFoundError: [Errno 2] No such file or directory: 'data.pk'